In [1]:
"""
url_c: course_url
"""
url_c = "https://www.mcu.edu.tw/student/new-query/sel-query/qslist_1.asp"
import requests
from bs4 import BeautifulSoup
import json
import re

#sch:校區
data = {"sch":2, "dept1":16}
c_res = requests.post(url_c, data=data)
c_res.encoding = 'big5-hkscs'
c_soup = BeautifulSoup(c_res.text, 'html.parser')


In [2]:
c_tr_tags = c_soup.find_all('tr')
c_idx_tags = c_tr_tags[0].find_all('td')

col_names = []
for name in c_idx_tags:
    col_names.append(name.text.strip())
col_names

['制別',
 '科目',
 '班級',
 '開班／選課人數',
 '任課教師',
 '上課日期／節次',
 '年級',
 '教室【校區】',
 '選別',
 '學分',
 '類別',
 '畢業班',
 '學期數',
 '說明']

In [22]:
col_keys = range(len(col_names))

In [24]:
data = dict(zip(col_keys,col_names))
data

{0: '制別',
 1: '科目',
 2: '班級',
 3: '開班／選課人數',
 4: '任課教師',
 5: '上課日期／節次',
 6: '年級',
 7: '教室【校區】',
 8: '選別',
 9: '學分',
 10: '類別',
 11: '畢業班',
 12: '學期數',
 13: '說明'}

In [32]:
for d in data:
    print(d, data[d])

0 制別
1 科目
2 班級
3 開班／選課人數
4 任課教師
5 上課日期／節次
6 年級
7 教室【校區】
8 選別
9 學分
10 類別
11 畢業班
12 學期數
13 說明


In [7]:
col_names = ['制別', '科目', '班級', '開班／選課人數', '任課教師', '上課日期／節次', '年級', '教室【校區】', '選別', '學分', '類別', '畢業班', '學期數', '說明']

In [13]:
col_names

['制別',
 '科目',
 '班級',
 '開班／選課人數',
 '任課教師',
 '上課日期／節次',
 '年級',
 '教室【校區】',
 '選別',
 '學分',
 '類別',
 '畢業班',
 '學期數',
 '說明']

In [64]:
col_names[0]

'制別'

In [382]:
course_data = []
for couse_tags in c_tr_tags[1:]:
    course_obj = {}
    i=0
    for c_col in couse_tags.find_all('td'):
        write_data = c_col.text.strip()
        if col_names[i] == "上課日期／節次":
            course_obj[ col_names[i] ] = clean_class_fmt(write_data)
        elif col_names[i] in ["班級","科目"]:
            course_obj[ col_names[i] ] = clean_space(write_data)
        else:
            course_obj[ col_names[i] ] = write_data
        i+=1
    course_data.append(course_obj)

fmt diff: 16151
fmt diff: 16152
fmt diff: 16251
fmt diff: 16252
fmt diff: 16253
fmt diff: 16351
fmt diff: 16352
fmt diff: 16353
fmt diff: 16354
fmt diff: 16355
fmt diff: 16356
fmt diff: 16357
fmt diff: 16358
fmt diff: 16451
fmt diff: 16551
fmt diff: 16552
fmt diff: 16553


In [371]:
def clean_class_fmt(class_string):
    ans = []
    if "\xa0" in class_string:
        l_split = class_string.split("\xa0")
        for s in l_split:
            data = s.replace(" ", "").replace("節","").replace("星期","").split(":")
            ans.append({"d":data[0],"t":data[1]})
    else:
        data = class_string.replace(" ", "").replace("節","").replace("星期","").split(":")
        ans.append({"d":data[0],"t":data[1]})
    return ans
def clean_space(class_string):
    data = class_string.split(" ")
    try:
        ans = {"id":data[0],"name":data[1]}
    except:
        ans = {"id":data[0]}
        print("fmt diff:", class_string)
    return ans
    

In [363]:
clean_space("16101 資傳一甲")

{'id': '16101', 'name': '資傳一甲'}

In [364]:
import pymongo

db_client = pymongo.MongoClient("mongodb://localhost:27017/")
if 'MCU_CourseDBdb' not in myclient.list_database_names():
    mydb = db_client["MCU_CourseDB"]
mycol = mydb["courses"]

if "courses" in mydb.list_collection_names():
    mycol.drop()
mycol.insert_many(course_data)

In [383]:
datas = mycol.find({}).limit(3)
for d in datas:
    print(d,"\n")

{'_id': ObjectId('5fe754eaa56d9763531e5132'), '制別': '大學部', '科目': {'id': '00121', 'name': '體育（壹）'}, '班級': {'id': '16101', 'name': '資傳一甲'}, '開班／選課人數': '75／70', '任課教師': '正課: 歐正聰', '上課日期／節次': [{'d': '1', 't': '0506'}], '年級': '1', '教室【校區】': '體育【 桃園】', '選別': '必修', '學分': '0', '類別': '', '畢業班': 'N', '學期數': '上學期', '說明': ''} 

{'_id': ObjectId('5fe754eaa56d9763531e5133'), '制別': '大學部', '科目': {'id': '00997', 'name': '班會'}, '班級': {'id': '16101', 'name': '資傳一甲'}, '開班／選課人數': '75／53', '任課教師': '正課: 李宇昇', '上課日期／節次': [{'d': '5', 't': '01'}], '年級': '1', '教室【校區】': 'S107【 桃園】', '選別': '必修', '學分': '0', '類別': '', '畢業班': 'N', '學期數': '全學年', '說明': ''} 

{'_id': ObjectId('5fe754eaa56d9763531e5134'), '制別': '大學部', '科目': {'id': '00999', 'name': '週會'}, '班級': {'id': '16101', 'name': '資傳一甲'}, '開班／選課人數': '70／53', '任課教師': ':', '上課日期／節次': [{'d': '5', 't': '05'}], '年級': '1', '教室【校區】': '【 桃園】', '選別': '必修', '學分': '0', '類別': '', '畢業班': 'N', '學期數': '全學年', '說明': ''} 



In [402]:
datas = mycol.find({"$and":[{"上課日期／節次.d":"2"},{"上課日期／節次.t":"05"}]}).limit(3)
for d in datas:
    print(d,"\n")

{'_id': ObjectId('5fe754eaa56d9763531e5139'), '制別': '大學部', '科目': {'id': '16116', 'name': '資訊概論'}, '班級': {'id': '16101', 'name': '資傳一甲'}, '開班／選課人數': '75／75', '任課教師': '正課: 共同指導', '上課日期／節次': [{'d': '2', 't': '05'}], '年級': '1', '教室【校區】': 'S107【 桃園】', '選別': '必修', '學分': '1', '類別': '', '畢業班': 'N', '學期數': '上學期', '說明': ''} 

{'_id': ObjectId('5fe754eaa56d9763531e5149'), '制別': '大學部', '科目': {'id': '00997', 'name': '班會'}, '班級': {'id': '16201', 'name': '資傳二甲'}, '開班／選課人數': '75／43', '任課教師': '正課: 陳惠惠', '上課日期／節次': [{'d': '2', 't': '05'}], '年級': '2', '教室【校區】': 'S106【 桃園】', '選別': '必修', '學分': '0', '類別': '', '畢業班': 'N', '學期數': '全學年', '說明': ''} 




class_obj
    course_id
    class_id
    time